# GG P2 Project

* ist199271 Margarida Estrela (33%)
  
* ist199891 André Teixeira (33%)
  
* ist1100096 Tiago Costa (33%)

Prof. Flávio Ramos (Prof. João Marques and Prof. Carlos Felgueiras in the labs)

Lab Shift number: PB02

## PART I – Original Database Schema (SQLite)

##### E-R Diagram

![E-R Diagram](soccer.drawio.png "E-R Diagram")

#### 1. The Original Database Schema

Study the E-R Diagram above corresponding to the original database schema.


Lets download the SQLite database backup/dump!

In [46]:
!wget -c https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download= -O /home/jovyan/data/database.sqlite

--2025-01-01 23:36:09--  https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download=
Resolving github.com (github.com)... 140.82.121.4
connected. to github.com (github.com)|140.82.121.4|:443... 
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/bdist/is-labs-data/refs/heads/main/soccer/database.sqlite?download=true [following]
--2025-01-01 23:36:09--  https://media.githubusercontent.com/media/bdist/is-labs-data/refs/heads/main/soccer/database.sqlite?download=true
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317698048 (303M) [application/octet-stream]
Saving to: ‘/home/jovyan/data/database.sqlite’

/home/jovyan/data/d 100%[===================>] 302.98M  11.2MB/s    i

#### 2. SQL Queries

Consider the E-R Diagram above and write SQL queries for each of the following information needs.

In [2]:
%reload_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql sqlite:////home/jovyan/data/database.sqlite --alias sqlite

1. What is the maximum number of goals recorded in a match? Output a single number.

In [13]:
%%sql sqlite
SELECT
    MAX(home_team_goal + away_team_goal) as max_goals
FROM
    match

max_goals
12



2. List each league name and its number of teams sorted from high to low.

In [18]:
%%sql sqlite
SELECT
    l.name,
    COUNT(DISTINCT t.team_api_id) AS num_teams
FROM
    match m
    JOIN
    league l ON m.league_id = l.id
    JOIN
    team t ON t.team_api_id IN (m.home_team_api_id, m.away_team_api_id)
GROUP BY
    l.name
ORDER BY
    num_teams DESC;


name,num_teams
France Ligue 1,35
England Premier League,34
Spain LIGA BBVA,33
Italy Serie A,32
Germany 1. Bundesliga,30
Portugal Liga ZON Sagres,29
Netherlands Eredivisie,25
Belgium Jupiler League,24
Poland Ekstraklasa,22
Scotland Premier League,17


3. Who are the GoalKeepers (GK) that have played all the away games for their team? Output the `team_long_name` and `player_name` (assuming `match.away_player_1` is the goalkeeper column).

In [20]:
%%sql sqlite
WITH matches_by_player AS (
    SELECT
        m.away_team_api_id,
        m.away_player_1,
        COUNT(*) AS player_match_count
    FROM
        match m
    GROUP BY
        m.away_team_api_id, m.away_player_1
),
total_matches AS (
    SELECT
        away_team_api_id,
        COUNT(*) AS total_match_count
    FROM
        match
    GROUP BY
        away_team_api_id
)
SELECT
    t.team_long_name,
    p.player_name
FROM
    matches_by_player mp
JOIN
    total_matches tm
    ON mp.away_team_api_id = tm.away_team_api_id
    AND mp.player_match_count = tm.total_match_count
JOIN
    team t
    ON t.team_api_id = mp.away_team_api_id
JOIN
    player p
    ON p.player_api_id = mp.away_player_1;



team_long_name,player_name
Fortuna Düsseldorf,Fabian Giefer
SV Darmstadt 98,Christian Mathenia
FC Energie Cottbus,Gerhard Tremmel
SC Paderborn 07,Lukas Kruse
RC Recreativo,Asier Riesgo
Watford,Heurelho Gomes
Dijon FCO,Baptiste Reynet
CD Tenerife,Sergio Aragones
DSC Arminia Bielefeld,Dennis Eilhoff


## PART II - New Database Schema (PostgreSQL)

#### 0. Migrate the SQLite Database to PostgreSQL

Instead of writing tedious migration code ourselves, we will be using the `pgloader` CLI tool to get started quickly. The migration task is setup using a `command-file`. Lets see inside that file.

In [3]:
!cat ./command-file.txt

load database
    from '/home/jovyan/data/database.sqlite'
    into postgresql://db:db@postgres/db

with include drop, create tables, create indexes, reset sequences,

prefetch rows = 100, max parallel create index = 1

set work_mem to '16MB', maintenance_work_mem to '512 MB';


The `pgloader` tool will take care of everything automatically including the foreign key constraints (which SQLite does not enforce by default). Lets run it.

In [55]:
!pgloader ./command-file.txt

2025-01-01T23:47:43.006000Z LOG pgloader version "3.6.10~devel"
2025-01-01T23:47:43.104000Z LOG Migrating from #<SQLITE-CONNECTION sqlite:///home/jovyan/data/database.sqlite {1008643E13}>
2025-01-01T23:47:43.104000Z LOG Migrating into #<PGSQL-CONNECTION pgsql://db@postgres:5432/db {1008644313}>
2025-01-01T23:47:52.808000Z LOG report summary reset
             table name     errors       rows      bytes      total time
-----------------------  ---------  ---------  ---------  --------------
                  fetch          0          0                     0.000s
        fetch meta data          0         50                     0.042s
         Create Schemas          0          0                     0.000s
       Create SQL Types          0          0                     0.003s
          Create tables          0         14                     0.047s
         Set Table OIDs          0          7                     0.005s
-----------------------  ---------  ---------  ---------  ---------

Lets take a closer look into the `match` table.

In [56]:
%load_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql postgresql+psycopg://db:db@postgres/db --alias psql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [57]:
%sqlcmd columns -t match

name,type,nullable,default,autoincrement,comment
id,BIGINT,False,nextval('match_id_seq'::regclass),True,None
country_id,BIGINT,True,None,False,None
league_id,BIGINT,True,None,False,None
season,TEXT,True,None,False,None
stage,BIGINT,True,None,False,None
date,TEXT,True,None,False,None
match_api_id,BIGINT,True,None,False,None
home_team_api_id,BIGINT,True,None,False,None
away_team_api_id,BIGINT,True,None,False,None
home_team_goal,BIGINT,True,None,False,None


Notice how `pgloader` even assigned the safest most-compatible data types for each column so that no data is lost in the migration.

##### 0.1. Introducing a limitation of the original database schema

The `match` table still does not look good to us... Can you try to write a SQL query for the following information need?

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [ ]:
%%sql psql
-- WILL NOT BE EVALUATED - We will be implementing a solution for this problem in the next section.

#### 1. Functions, Stored Procedures, and Triggers

Provide the SQL instructions corresponding to each of the following tasks:

1. Create a new `match_player` table to store the data for the starting players (11) for each match (currently stored in the columns `match.home_player_DD` and `match.away_player_DD`). We are going to copy the data contained in the table `match` to `match_player` but we want to use a new column for the player position `position`.

In [ ]:
%%sql psql
DROP TABLE IF EXISTS match_player;

CREATE TABLE match_player (
    player_api_id INTEGER , 
    match_api_id INTEGER ,
    team_api_id INTEGER ,
    position VARCHAR(20) ,
    home_team boolean NOT NULL,
    PRIMARY KEY (player_api_id, match_api_id, position),
    FOREIGN KEY (team_api_id) REFERENCES team(team_api_id) ON DELETE CASCADE,
    FOREIGN KEY (player_api_id) REFERENCES player(player_api_id) ON DELETE CASCADE,
    FOREIGN KEY (match_api_id) REFERENCES match(match_api_id) ON DELETE CASCADE
);

2. Populate the new table `match_player` using the data contained in the table `match` using a function or stored procedure.

In [ ]:
%%sql psql
DELETE FROM match_player;
CREATE OR REPLACE PROCEDURE populate_match_player()
LANGUAGE plpgsql
AS $$
BEGIN
    -- Insert for home team players
    INSERT INTO match_player (player_api_id, match_api_id, team_api_id, position, home_team)
    SELECT DISTINCT
        player_id,
        match_api_id,
        team_api_id,
        position,
        home_team
    FROM (
        SELECT 
            unnest(ARRAY[
                home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, 
                home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, 
                home_player_11
            ]) AS player_id,
            match_api_id,
            home_team_api_id AS team_api_id,
            unnest(ARRAY[
                'home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 
                'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 
                'home_player_11'
            ]) AS position,
            TRUE AS home_team
        FROM match
    ) AS subquery
    WHERE player_id IS NOT NULL
      AND player_id::TEXT <> 'None';

    -- Insert for away team players
    INSERT INTO match_player (player_api_id, match_api_id, team_api_id, position, home_team)
    SELECT DISTINCT
        player_id,
        match_api_id,
        team_api_id,
        position,
        home_team
    FROM (
        SELECT 
            unnest(ARRAY[
                away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, 
                away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, 
                away_player_11
            ]) AS player_id,
            match_api_id,
            away_team_api_id AS team_api_id,
            unnest(ARRAY[
                'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 
                'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 
                'away_player_11'
            ]) AS position,
            FALSE AS home_team
        FROM match
    ) AS subquery
    WHERE player_id IS NOT NULL
      AND player_id::TEXT <> 'None';
END;
$$;

-- Call the procedure
CALL populate_match_player();

#### 2. SQL Queries

Consider the new `match_player` table above and write SQL queries for each of the following information needs.

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [ ]:
%%sql psql
SELECT DISTINCT t.team_short_name, t.team_long_name
FROM match_player m
JOIN team t ON t.team_api_id = m.team_api_id
JOIN player p ON p.player_api_id = m.player_api_id 
WHERE p.player_name = 'Cristiano Ronaldo'


2. Who are the players that have played all the away games for their team? Output the `team_long_name` and `player_name`.

In [ ]:
%%sql psql
WITH PlayerAwayGames AS (
    SELECT 
        mp.player_api_id, 
        t.team_api_id, 
        COUNT(mp.match_api_id) AS player_away_games
    FROM match_player mp
    JOIN team t ON mp.team_api_id = t.team_api_id
    WHERE  mp.home_team = false
    GROUP BY 
        mp.player_api_id, 
        t.team_api_id
),
TeamAwayGames AS (
    SELECT 
        t.team_api_id, 
        COUNT(m.away_team_api_id) AS team_away_games
    FROM match m
    JOIN team t ON m.away_team_api_id = t.team_api_id
    GROUP BY 
        t.team_api_id
)
SELECT 
    t.team_long_name, 
    p.player_name
FROM PlayerAwayGames pag
JOIN TeamAwayGames tag ON pag.team_api_id = tag.team_api_id
JOIN player p ON pag.player_api_id = p.player_api_id
JOIN Team t on pag.team_api_id = t.team_api_id
WHERE pag.player_away_games = tag.team_away_games
ORDER BY 
    pag.team_api_id, 
    pag.player_api_id;

3. What is the match with the highest number of goals? Output `team_long_name` for both home and away teams, and the total number of goals for each team.

In [ ]:
%%sql psql
WITH game_goals AS (
    SELECT
        match_api_id,
        home_team_goal + away_team_goal AS total_goals
    FROM
        match
),
max_goals AS (
    SELECT
        MAX(total_goals) AS max_total_goals
    FROM
        game_goals
)
SELECT
    m.match_api_id,
    home_team.team_long_name AS home_team_name,
    m.home_team_goal AS home_team_goals,
    away_team.team_long_name AS away_team_name,
    m.away_team_goal AS away_team_goals,
    g.total_goals
FROM
    game_goals g
JOIN match m ON g.match_api_id = m.match_api_id
JOIN team home_team ON m.home_team_api_id = home_team.team_api_id
JOIN team away_team ON m.away_team_api_id = away_team.team_api_id
WHERE
    g.total_goals = (SELECT max_total_goals FROM max_goals);

#### 3. Query Optimization

Consider the new `player_attributes` table that contains the largest amount of rows.

1. Create the most appropriate index(es) for the following query: 

In [ ]:
%%sql psql
DROP INDEX IF EXISTS idx_player_attributes_attacking_work_rate_rating;

CREATE INDEX idx_player_attributes_attacking_work_rate_rating
ON player_attributes (attacking_work_rate, overall_rating DESC);

In [ ]:
%%sql psql
SELECT
  MAX(overall_rating)
FROM
  player_attributes
WHERE
  attacking_work_rate = 'high';

2. Create the most appropriate index(es) for the following query: 

In [ ]:
%%sql psql
CREATE INDEX
DROP INDEX IF EXISTS idx_player_attributes_group_having_partial, idx_player_attributes_group_having_refined;
    
--CREATE INDEX idx_player_attributes_group_having_refined
--ON player_attributes (player_api_id, preferred_foot, crossing, potential);
CREATE INDEX idx_player_attributes_group_having_partial
ON player_attributes (player_api_id, preferred_foot, crossing, potential)
WHERE crossing = 80 AND potential >= 90;

In [ ]:
%%sql psql
SELECT
  player_api_id,
  preferred_foot,
  MIN(crossing),
  MIN(potential)
FROM
  player_attributes
GROUP BY
  player_api_id,
  preferred_foot
HAVING
  MIN(crossing) = 80
  AND MIN(potential) >= 90

## PART III - The Web App (Flask + PostgreSQL)

#### 1. SQL View

1. Create a SQL View that lists the player names sorted by the most recent `player_attributes.date` first.

In [92]:
%%sql psql
DROP VIEW IF EXISTS players_index_view;

CREATE VIEW players_index_view AS
SELECT p.player_api_id, p.player_name, pa.max_date AS date
FROM 
    player p
JOIN (
    SELECT player_api_id, MAX(date) AS max_date
    FROM player_attributes
    GROUP BY player_api_id
) pa
    ON p.player_api_id = pa.player_api_id
ORDER BY pa.max_date DESC;

++
||
++
++

2. Create the most appropriate index(es) for the SQL view you just created. (Note: PostgreSQL does not have support for materialized views)

In [97]:
%%sql psql

DROP INDEX IF EXISTS player_api_id_date_idx;
DROP INDEX IF EXISTS player_api_id_p_idx;
    
CREATE INDEX player_api_id_date_idx ON player_attributes (player_api_id, date DESC);
CREATE INDEX player_api_id_p_idx ON player (player_api_id);

++
||
++
++

3. Please test your view using the following query.

In [98]:
%%sql psql
SELECT
  *
FROM
  players_index_view
LIMIT
  20;

player_name,date
Baptiste Aloe,2025-01-21 16:16:28
Yanis Mbombo Lokwa,2016-07-07 00:00:00
Adalberto Penaranda,2016-07-07 00:00:00
Ronnie Schwartz,2016-07-07 00:00:00
Florian Lejeune,2016-07-07 00:00:00
Bebe,2016-07-07 00:00:00
Kevin Koubemba,2016-07-07 00:00:00
Wallace,2016-07-07 00:00:00
Vasyl Pryima,2016-06-30 00:00:00
Eoin Doyle,2016-06-30 00:00:00


#### 2. Create the players_index page on the Web App

Requirements:
- Should use the SQL View declared above
- Do not list more than 20 players.

1. Copy your players_index function from app.py to the cell below

```python
@app.route("/", methods=("GET",))
@app.route("/players", methods=("GET",))
@limiter.limit("1 per second")
def players_index():
    """Show all the players, most recent date first."""

    with pool.connection() as conn:
        with conn.cursor() as cur:
            players = cur.execute(
                """
                SELECT p.player_api_id, p.player_name, pa.max_date AS date
                FROM player p
                JOIN (
                    SELECT player_api_id, MAX(date) AS max_date
                    FROM player_attributes
                    GROUP BY player_api_id
                ) pa ON p.player_api_id = pa.player_api_id
                ORDER BY pa.max_date DESC
                LIMIT 20;
                """,
                {},
            ).fetchall()
            log.debug(f"Found {cur.rowcount} rows.")

    return render_template("players/index.html", players=players)
```

2. Copy your final HTML template for this view onto the cell below

```html
{% extends "base.html" %}
{% block header %}
    <h1>
        {% block title %}
            Players
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}
    {% for player in players %}
        <article class="post">
            <header>
                <div>
                    <h1>{{ player['player_name'] }}</h1>
                    <div class="about">API ID {{ player['player_api_id'] }}</div>
                </div>
                <a class="action"
                    href="{{ url_for('player_update_view', player_api_id=player['player_api_id']) }}">Edit</a>
            </header>
            <p class="body">Date: {{ player['date'] }}</p>
        </article>
        {% if not loop.last %}<hr />{% endif %}
    {% endfor %}
{% endblock content %}
```

#### 3. Create the player_update_view and player_update_save routes on the Web App

Requirements:

- The app.route for player_update_view should take a player_api_id and show a nice editable HTML form.
             
- The app.route for player_update_save should be called on form submission and insert the new data into player_attributes. 

- On success, should redirect to the players_index

- On failure, should raise an appropriate exception with a error message

1. Copy your player_update_view function from app.py to the cell below

```python
@app.route("/players/<player_api_id>/update", methods=("GET",))
@limiter.limit("1 per second")
def player_update_view(player_api_id):
    """Show the page to update the player date."""

    with pool.connection() as conn:
        with conn.cursor() as cur:
            player = cur.execute(
                """
                SELECT player_api_id
                FROM player_attributes
                WHERE player_api_id = %(player_api_id)s;
                """,
                {"player_api_id": player_api_id},
            ).fetchone()
            log.debug(f"Found {cur.rowcount} rows.")

    # At the end of the `connection()` context, the transaction is committed
    # or rolled back, and the connection returned to the pool.

    return render_template("players/update.html", player=player)
```

2. Copy your final HTML template for this view onto the cell below

```html
{% extends "base.html" %}
{% block header %}
    <h1>
        {% block title %}
            Player {{ player['player_api_id'] }} | Edit
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}
    <form method="post">
        <label for="player_api_id">Player API ID</label>
        <input name="player_api_id"
                id="player_api_id"
                type="text"
                value="{{ request.form['player_api_id'] or player['player_api_id'] }}"
                disabled />
        <label for="date">Date</label>
        <input name="date"
                id="date"
                type="datetime-local"
                step="1"
                value="{{ request.form['date'] or player['date'] }}"/>
        <input type="submit" value="Save" />
    </form>
{% endblock content %}
```

3. Copy your player_update_save function from app.py to the cell below

```python
@app.route("/players/<player_api_id>/update", methods=("POST",))
def player_update_save(player_api_id):
    """Update the player date."""

    date = request.form["date"]

    error = None

    if date == "":
        error = "Date is required."
    elif not is_valid_date(date):
        error = "Date is required to be in the format 'DD/MM/YYYY, HH:MM:SS'."

    if error is not None:
        flash(error)
    else:
        date = date.replace("T", " ")   # Replace 'T' with a space
        with pool.connection() as conn:
            with conn.cursor() as cur:
                cur.execute(
                    """
                    UPDATE player_attributes
                    SET date = %(date)s
                    WHERE player_api_id = %(player_api_id)s;
                    """,
                    {"player_api_id": player_api_id, "date": date},
                )
                # The result of this statement is persisted immediately by the database
                # because the connection is in autocommit mode.

        # The connection is returned to the pool at the end of the `connection()` context but,
        # because it is not in a transaction state, no COMMIT is executed.

    return redirect(url_for("players_index"))
```